In [1]:
%pip install -q gpt-2-simple
import gpt_2_simple as gpt2

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
MODEL_NAME='355M'
FILE_NAME='en.macron.txt'
RUN_NAME='run1'
PREFIX='My'
sess = gpt2.start_tf_sess()

In [3]:
gpt2.download_gpt2(model_name=MODEL_NAME)

Fetching checkpoint: 1.05Mit [00:00, 735Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 9.81Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 1.21Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:34, 40.9Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 1.10Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 7.97Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 10.5Mit/s]                                                      


In [4]:
gpt2.finetune(
    sess,
    dataset=FILE_NAME,
    model_name=MODEL_NAME,
    steps=1000,
    restore_from='fresh',
    run_name=RUN_NAME,
    print_every=10,
    sample_every=200,
    save_every=500,
    learning_rate=1e-5
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


INFO:tensorflow:Restoring parameters from models/355M/model.ckpt
  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  3.34it/s]


dataset has 35464 tokens
Training...
[10 | 22.48] loss=3.46 avg=3.46
[20 | 36.49] loss=3.52 avg=3.49
[30 | 50.54] loss=3.33 avg=3.43
[40 | 64.71] loss=3.26 avg=3.39
[50 | 78.98] loss=3.58 avg=3.43
[60 | 93.35] loss=3.10 avg=3.37
[70 | 107.61] loss=3.56 avg=3.40
[80 | 121.73] loss=3.37 avg=3.40
[90 | 135.79] loss=3.28 avg=3.38
[100 | 149.78] loss=3.12 avg=3.35
[110 | 163.77] loss=3.23 avg=3.34
[120 | 177.79] loss=3.52 avg=3.36
[130 | 191.79] loss=3.16 avg=3.34
[140 | 205.77] loss=2.89 avg=3.31
[150 | 219.79] loss=3.05 avg=3.29
[160 | 233.83] loss=3.18 avg=3.28
[170 | 247.90] loss=2.82 avg=3.25
[180 | 262.03] loss=2.96 avg=3.23
[190 | 276.17] loss=2.70 avg=3.20
[200 | 290.33] loss=2.94 avg=3.19
======== SAMPLE 1 ========
 France will be very good for Germany. And it does not require too much thought when looking at this subject, for we have a common European ideal that is, of course, even here and in France. But in principle, it is up to each nation to decide for itself. No decision. On 

Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [12]:
results = gpt2.generate(
    sess,
    length=100,
    temperature=.7,
    prefix=PREFIX,
    nsamples=5,
    run_name=RUN_NAME,
    return_as_list=True
)
print(results)

['My dear sister, we are at war. War has struck us as a nation and it is up to us to fight against it and to overcome it. We will not give up. We will not give in to defeat. My dear compatriots, our thoughts and our prayers are with the loved ones of the fallen at this time. The government, the law enforcers, the armed forces, the civil service, the media, all these are at war. All are engaged in this war against the spread', 'My dear Americans, tonight I want to tell you that I have been thinking a lot about you recently. I saw it on your faces as I have been, a sense of urgency and urgency to do, of tomorrow. My dear compatriots, we have not only survived, we have survived because we have committed ourselves. Because we know that when we work together, we succeed. Because we are America, we are union, we are the strongest economy in Europe, we are sovereign, we are solidarity. Because', "My dear compatriots, for the past six months we've been working hard to bring about a political s

In [13]:
import boto3

translator = boto3.client('translate')
for res in results:
    translation = translator.translate_text(Text=res, SourceLanguageCode='en', TargetLanguageCode='fr')
    text = translation.get('TranslatedText').rsplit(".", 1)
    print(text[0], end='.\n\n')

Ma chère sœur, nous sommes en guerre. La guerre nous a frappés en tant que nation et c'est à nous de la combattre et de la surmonter. Nous n'abandonnerons pas. Nous ne céderons pas à la défaite. Mes chers compatriotes, nos pensées et nos prières sont avec les proches des déchus en ce moment. Le gouvernement, les forces de l'ordre, les forces armées, la fonction publique, les médias, tout cela est en guerre.

Mes chers Américains, ce soir, je veux vous dire que j'ai beaucoup réfléchi à vous récemment. Je l'ai vu sur vos visages comme je l'ai été, un sentiment d'urgence et d'urgence à faire, de demain. Mes chers compatriotes, non seulement nous avons survécu, nous avons survécu parce que nous nous sommes engagés nous-mêmes. Parce que nous savons que lorsque nous travaillons ensemble, nous réussissons. Parce que nous sommes l'Amérique, nous sommes l'union, nous sommes l'économie la plus forte d'Europe, nous sommes souverains, nous sommes solidaires.

Mes chers compatriotes, depuis six moi

In [9]:
for temperature in [.2, .5, .7, 1, 2]:
    results = gpt2.generate(
        sess,
        length=50,
        temperature=temperature,
        prefix=PREFIX,
        nsamples=1,
        run_name=RUN_NAME,
        return_as_list=True
    )
    translation =translator.translate_text(Text=results[0], SourceLanguageCode='en', TargetLanguageCode='fr')
    text = translation.get('TranslatedText').rsplit(".", 1)
    print(f'{temperature}: {text[0]}', end='.\n\n')

0.2: Mes chers compatriotes, nous sommes en guerre. Je sais que certains d'entre vous ont pensé à cette guerre. Il est important pour nous, en tant que nation, de nous unir.

0.5: Mes chers compatriotes, je suis conscient que la période de turbulence et de tension que nous traversons ne se sent pas bien pour nous.

0.7: Mon cher frère, chaque jour, nous progressons contre le virus. Nous avons déjà enregistré plus de 1000 nouveaux diagnostics, principalement en région parisienne, Marseille, Lyon et Bruxelles.

1: Mes chers Cataloniens, secouer par nos menaces et nos événements de ces derniers jours. Vous l'avez vu dans les images, le son, la fureur, la passion des champions, le dévouement, la volonté de réussir qui est devenue virale.

2: Mon cher Drirez Induti Zimmerman.. Alors que Gujarati sida approché 7000.>> Les patients probablement libre action de masse prochaine série sloganuku quatorze quarante.

